# A Simple Scraper For Collecting Images Found Using Google Image Search

**How To Use:** First, manually copy the body element via Chrome's Inspect feature into a text document and save it in the notebook directory; include the path to this file in the `searches` parameter below. Running this code should collect all the images associated with each search, and place them in a folder whose name matches that of your search.

In [1]:
# scraping dependencies
import scrapy
from scrapy.item import Item, Field
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from scrapy.pipelines.images import ImagesPipeline
import ast
from tqdm import tqdm

In [2]:
# parameters
#searches = ['auditoriums', 'beaches', 'caves', 'cities', 'deserts', 'livingrooms', 'suburbs']
searches = ['suburb']
targetdirectory = '/Users/jordangunn/Dropbox/imagescrape'

In [3]:
# classes defining spider
class ImageItem(scrapy.Item):
    image_urls = scrapy.Field()
    search = scrapy.Field()
    
class ImageSpider(scrapy.Spider):
    name = 'googleimagespider'
    custom_settings = {'ITEM_PIPELINES': {'scrapy.pipelines.images.ImagesPipeline': 1},
                       'IMAGES_STORE' : targetdirectory,
                      'LOG_LEVEL': 'WARNING'}
    
    def start_requests(self):
        baseurl = 'https://www.google.com/'
        yield scrapy.Request(baseurl, callback=self.parse)
        
    def parse(self, response):
        for search in searches:
            with open(search) as f:
                a = Selector(text=f.read())
            image = ImageItem()
            image['search'] = search
            image['image_urls'] = [ast.literal_eval(each.extract())['ou'] for each in a.xpath('//*[@id="rg_s"]/div/div/text()')]
            yield image

In [ ]:
# code starting scrape
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
process.crawl(ImageSpider)
process.start()